## IA PROYECT


# 1. Implementamos librerias, configuración del Entorno y Carga de Datos

In [1]:
# 1. Instalar librería de interpretabilidad SHAP
!pip install shap

# 2. Importar módulos esenciales
import pandas as pd
import numpy as np
import tensorflow as tf
from google.colab import drive

# Módulos de Scikit-learn
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score

# Módulo SHAP
import shap

In [2]:
# 3. Montar Google Drive y Cargar el Dataset
# **¡ATENCIÓN! REVISA Y PEGA LA RUTA CORRECTA DE TU ARCHIVO EN DRIVE**
try:
    drive.mount('/content/drive')
    ruta_dataset = '/content/drive/MyDrive/IAProyecto/heart_disease_uci.csv' ## DEBE SER CAMBIADO SEGÚN LA RUTA DONDE FUE AGREGADA EL .CSV!!!
    df = pd.read_csv(ruta_dataset)
    print("ÉXITO: Dataset cargado correctamente. Dimensiones:", df.shape)
except FileNotFoundError:
    print("ERROR FATAL: Archivo no encontrado. Revisa la ruta en 'ruta_dataset'.")
except Exception as e:
    print(f"ERROR AL CARGAR: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ÉXITO: Dataset cargado correctamente. Dimensiones: (920, 16)


## 2. Procesamiento robusto

In [3]:
# 1. Reemplazar valores erróneos ('?') por NaN y eliminar columnas irrelevantes
df.replace('?', np.nan, inplace=True)
df.drop(columns=['id', 'dataset'], inplace=True)

# 2. Identificar columnas con posibles errores de tipo (como 'ca' y 'thal' que pueden ser object/string por los '?' )
for col in ['ca', 'thalch', 'oldpeak', 'trestbps', 'chol']:
    df[col] = pd.to_numeric(df[col], errors='coerce') # Forzar a numérico, convirtiendo errores a NaN

# 3. Imputación de nulos
for col in df.columns:
    if df[col].dtype == object:
        df[col].fillna(df[col].mode()[0], inplace=True) # Moda para categóricos
    elif df[col].isnull().any():
        df[col].fillna(df[col].median(), inplace=True) # Mediana para numéricos

# 4. Conversión de Target a Binario y Codificación
df['target'] = df['num'].apply(lambda x: 1 if x > 0 else 0)
df.drop(columns=['num'], inplace=True)

# 5. Codificación One-Hot
df = pd.get_dummies(df, drop_first=True)

# 6. Definición X e Y
y = df['target']
X = df.drop(columns=['target'])
feature_names = X.columns

/tmp/ipython-input-4173791650.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True) # Moda para categóricos
/tmp/ipython-input-4173791650.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(valu

## 3. División, Estandarización y preparación final

In [4]:
# 1. División (70% Entrenamiento, 30% Prueba)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

# 2. Estandarización de Features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convertir de vuelta a DataFrame (necesario para SHAP)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_names)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_names)

print("Paso 3 completado. Los datos están listos y estandarizados.")

Paso 3 completado. Los datos están listos y estandarizados.


## 4. Entrenamiento y Evaluación de Modelos ML (Random Forest/SVC)

In [5]:
# 1. Inicializar y Entrenar Random Forest
rf_model = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# 2. Evaluación Random Forest
y_pred_rf = rf_model.predict(X_test_scaled)
y_prob_rf = rf_model.predict_proba(X_test_scaled)[:, 1]

print(f"\n--- Random Forest (ML) Resultados ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"Sensibilidad (Recall): {recall_score(y_test, y_pred_rf):.4f}")
print(f"AUC: {roc_auc_score(y_test, y_prob_rf):.4f}")

# 3. Entrenamiento y Evaluación SVM (Ejemplo de otro modelo requerido)
svc_model = SVC(kernel='rbf', probability=True, random_state=42)
svc_model.fit(X_train_scaled, y_train)

y_prob_svc = svc_model.predict_proba(X_test_scaled)[:, 1]
print(f"\n--- SVC (ML) Resultados ---")
print(f"AUC: {roc_auc_score(y_test, y_prob_svc):.4f}")


--- Random Forest (ML) Resultados ---
Accuracy: 0.8333
Sensibilidad (Recall): 0.8562
AUC: 0.8984

--- SVC (ML) Resultados ---
AUC: 0.9091


## 5. Implementación de Modelo DL (Red Neuronal Profunda)

In [6]:
# 1. Definición de la Arquitectura DL
input_dim = X_train_scaled.shape[1]

dl_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(input_dim,)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# 2. Compilación y Entrenamiento
dl_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

dl_model.fit(
    X_train_scaled, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test_scaled, y_test),
    verbose=0
)

# 3. Evaluación final
loss_dl, accuracy_dl = dl_model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"\n--- Deep Learning (DL) Resultados ---")
print(f"Accuracy: {accuracy_dl:.4f}")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



--- Deep Learning (DL) Resultados ---
Accuracy: 0.8188


## 6. Análisis de Interpretabilidad (SHAP)

In [7]:
# 1. Seleccionar el modelo de mejor desempeño (Usaremos Random Forest como ejemplo)
best_model = rf_model

# 2. Crear el objeto Explainer
# Nota: Si usas DL, el explainer cambia (shap.KernelExplainer o DeepExplainer)
explainer = shap.TreeExplainer(best_model)

# 3. Calcular valores SHAP (para la clase 'Enfermo' = 1)
shap_values = explainer.shap_values(X_test_scaled)

print("\n--- Análisis SHAP para Justificación Clínica ---")
print("El modelo está listo para generar gráficos de justificación")


--- Análisis SHAP para Justificación Clínica ---
El modelo está listo para generar gráficos de justificación


In [8]:
# Importar las librerías necesarias para las métricas
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, confusion_matrix
import pandas as pd

# 1. Función para calcular todas las métricas requeridas
def get_all_metrics(model, X_data, y_data, model_type='ML', threshold=0.5):
    """
    Calcula un conjunto completo de métricas para un modelo y un conjunto de datos.
    """
    # Predicciones de clase (0 o 1)
    if model_type == 'ML':
        y_pred = model.predict(X_data)
        y_prob = model.predict_proba(X_data)[:, 1]  # Probabilidades de la clase positiva
    else:  # Modelo de Deep Learning (TensorFlow/Keras)
        y_prob = model.predict(X_data, verbose=0).flatten()
        y_pred = (y_prob > threshold).astype(int)  # Ajustar umbral de decisión

    # Cálculo de métricas
    accuracy = accuracy_score(y_data, y_pred)
    recall = recall_score(y_data, y_pred, average='binary')  # Para clasificación binaria
    auc = roc_auc_score(y_data, y_prob)  # AUC para clasificación binaria

    # Para la Especificidad, necesitamos la matriz de confusión
    tn, fp, fn, tp = confusion_matrix(y_data, y_pred).ravel()
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    # Devolver resultados formateados para la tabla
    return [f"{accuracy*100:.2f}%", f"{recall*100:.2f}%", f"{specificity*100:.2f}%", f"{auc:.3f}"]

# --- OBTENCIÓN DE MÉTRICAS COMPARATIVAS ---
# Asegúrate de que los modelos 'rf_model' y 'dl_model' ya han sido entrenados
# y que los datos X_train_scaled, y_train, X_test_scaled, y_test están disponibles.

print("Evaluando el rendimiento de los modelos...")

# 2. Evaluación en el CONJUNTO DE PRUEBA (datos no vistos)
rf_test_results = get_all_metrics(rf_model, X_test_scaled, y_test, model_type='ML')
dl_test_results = get_all_metrics(dl_model, X_test_scaled, y_test, model_type='DL')

# 3. Evaluación en el CONJUNTO DE ENTRENAMIENTO (para detectar sobreajuste)
rf_train_results = get_all_metrics(rf_model, X_train_scaled, y_train, model_type='ML')
dl_train_results = get_all_metrics(dl_model, X_train_scaled, y_train, model_type='DL')

# --- CREACIÓN DE LA TABLA COMPARATIVA FINAL ---

# 4. Organizar los datos en un diccionario
final_data = {
    'Métrica': ['Precisión (Accuracy)', 'Sensibilidad (Recall)', 'Especificidad (Specificity)', 'AUC'],
    'RF en Entrenamiento': rf_train_results,
    'RF en Prueba': rf_test_results,
    'Red Neuronal en Entrenamiento': dl_train_results,
    'Red Neuronal en Prueba': dl_test_results
}

# 5. Crear y mostrar el DataFrame en formato Markdown
df_final_results = pd.DataFrame(final_data)

print("\n--- TABLA DE RESULTADOS COMPARATIVOS (ENTRENAMIENTO VS. PRUEBA) ---")
print(df_final_results.to_markdown(index=False))

Evaluando el rendimiento de los modelos...

--- TABLA DE RESULTADOS COMPARATIVOS (ENTRENAMIENTO VS. PRUEBA) ---
| Métrica                     | RF en Entrenamiento   | RF en Prueba   | Red Neuronal en Entrenamiento   | Red Neuronal en Prueba   |
|:----------------------------|:----------------------|:---------------|:--------------------------------|:-------------------------|
| Precisión (Accuracy)        | 100.00%               | 83.33%         | 96.27%                          | 81.88%                   |
| Sensibilidad (Recall)       | 100.00%               | 85.62%         | 96.63%                          | 86.27%                   |
| Especificidad (Specificity) | 100.00%               | 80.49%         | 95.83%                          | 76.42%                   |
| AUC                         | 1.000                 | 0.898          | 0.995                           | 0.883                    |


EVALUACIÓN


In [9]:

from sklearn.metrics import accuracy_score, recall_score, confusion_matrix

def calculate_metrics(y_true, y_pred):
    """Calcula Precisión (Accuracy), Sensibilidad (Recall) y Especificidad."""
    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred, zero_division=0)

    # Se extraen TP, FP, TN, FN de la Matriz de Confusión para calcular Especificidad
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    # Especificidad = TN / (TN + FP)
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0

    return f"{accuracy * 100:.2f}%", f"{recall * 100:.2f}%", f"{specificity * 100:.2f}%"

print("Función 'calculate_metrics' definida.")

Función 'calculate_metrics' definida.


In [10]:
# --- 1. CONFIGURACIÓN INICIAL Y PREPROCESAMIENTO ---
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC # Importado en el notebook original

# Carga de datos (ajusta la ruta si es necesario)
df = pd.read_csv('/content/drive/MyDrive/IAProyecto/heart_disease_uci.csv')

# Preprocesamiento robusto (similar al Paso 2 de tu notebook)
df.replace('?', np.nan, inplace=True)
df.drop(columns=['id', 'dataset'], inplace=True)

for col in ['ca', 'thal']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

for col in df.columns:
    if df[col].dtype == object or col in ['cp', 'restecg', 'slope']:
        df[col].fillna(df[col].mode()[0], inplace=True)
    elif df[col].isnull().any():
        df[col].fillna(df[col].median(), inplace=True)

# Conversión de Target a Binario y Codificación One-Hot
df['target'] = df['num'].apply(lambda x: 1 if x > 0 else 0)
df.drop(columns=['num'], inplace=True)
df = pd.get_dummies(df, drop_first=True)

y = df['target']
X = df.drop(columns=['target'])
feature_names = X.columns.tolist()

# División y Estandarización (Paso 3)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_names)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_names)

# --- 2. ENTRENAMIENTO DE MODELOS FINALES (RF y DL) ---
# Random Forest (Corregido: min_samples_leaf=5)
print("Entrenando Random Forest Final...")
rf_model_final = RandomForestClassifier(
    n_estimators=100, max_depth=5, min_samples_leaf=5, random_state=42
)
rf_model_final.fit(X_train_scaled, y_train)

# Red Neuronal Profunda (Corregida: con Dropout)
print("Entrenando Red Neuronal Final...")
input_dim = X_train_scaled.shape[1]
tf.random.set_seed(42)
np.random.seed(42)

nn_model_final = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(input_dim,)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

nn_model_final.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
nn_model_final.fit(X_train_scaled, y_train, epochs=100, batch_size=32, verbose=0)

print("Entrenamiento completado.")

Entrenando Random Forest Final...


/tmp/ipython-input-4202372472.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)
/tmp/ipython-input-4202372472.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

Entrenando Red Neuronal Final...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Entrenamiento completado.


In [11]:
# Predicciones Random Forest
y_pred_train_rf = rf_model_final.predict(X_train_scaled)
y_pred_test_rf = rf_model_final.predict(X_test_scaled)

# Predicciones Red Neuronal
y_pred_train_nn_prob = nn_model_final.predict(X_train_scaled, verbose=0)
y_pred_test_nn_prob = nn_model_final.predict(X_test_scaled, verbose=0)

# Convertir probabilidades (Red Neuronal) a clases (0 o 1)
y_pred_train_nn = (y_pred_train_nn_prob > 0.5).astype(int).flatten()
y_pred_test_nn = (y_pred_test_nn_prob > 0.5).astype(int).flatten()

# Calcular métricas para la tabla
rf_train_metrics = calculate_metrics(y_train, y_pred_train_rf)
rf_test_metrics = calculate_metrics(y_test, y_pred_test_rf)
nn_train_metrics = calculate_metrics(y_train, y_pred_train_nn)
nn_test_metrics = calculate_metrics(y_test, y_pred_test_nn)

print("Métricas calculadas para los 4 escenarios (2 modelos x 2 datasets).")

Métricas calculadas para los 4 escenarios (2 modelos x 2 datasets).


In [12]:
metrics = ["Precisión (Accuracy)", "Sensibilidad (Recall)", "Especificidad (Specificity)"]

results = {
    "Métrica": metrics,
    "RF Final (Entrenamiento)": rf_train_metrics,
    "RF Final (Prueba)": rf_test_metrics,
    "Red Neuronal Final (Entrenamiento)": nn_train_metrics,
    "Red Neuronal Final (Prueba)": nn_test_metrics
}

df_results = pd.DataFrame(results)

# Mostrar la tabla en formato Markdown
print("\n--- TABLA DE RESULTADOS DEFINITIVA (EVALUACIÓN) ---")
print(df_results.to_markdown(index=False))



--- TABLA DE RESULTADOS DEFINITIVA (EVALUACIÓN) ---
| Métrica                     | RF Final (Entrenamiento)   | RF Final (Prueba)   | Red Neuronal Final (Entrenamiento)   | Red Neuronal Final (Prueba)   |
|:----------------------------|:---------------------------|:--------------------|:-------------------------------------|:------------------------------|
| Precisión (Accuracy)        | 86.96%                     | 80.43%              | 44.72%                               | 44.57%                        |
| Sensibilidad (Recall)       | 91.85%                     | 83.66%              | 0.00%                                | 0.00%                         |
| Especificidad (Specificity) | 80.90%                     | 76.42%              | 100.00%                              | 100.00%                       |


In [13]:
import pandas as pd
import numpy as np

def predict_and_interpret_patient(model, scaler, feature_columns, raw_data):
    df_new = pd.DataFrame([raw_data])
    df_pred = pd.get_dummies(df_new, drop_first=True)
    df_pred = df_pred.reindex(columns=feature_columns, fill_value=0)
    data_scaled = scaler.transform(df_pred)
    df_scaled = pd.DataFrame(data_scaled, columns=feature_columns)
    pred_class = model.predict(df_scaled)[0]
    pred_prob = model.predict_proba(df_scaled)[0, 1]
    if pred_prob >= 0.70:
        interpretacion = "ALTO RIESGO (Acción Clínica Recomendada)"
        justificacion = "La probabilidad supera el 70%, indicando un patrón clínico severo y consistente con la enfermedad."
    elif pred_prob >= 0.50:
        interpretacion = "RIESGO SIGNIFICATIVO (Monitoreo Urgente)"
        justificacion = "La probabilidad es alta, requiriendo validación médica inmediata. El modelo está muy cerca del umbral de clasificación."
    elif pred_prob >= 0.40:
        interpretacion = "RIESGO MODERADO (Seguimiento Cercano)"
        justificacion = "La probabilidad está cerca del 50%, lo que sugiere que el paciente tiene factores de riesgo que deben ser monitoreados."
    else:
        interpretacion = "BAJO RIESGO (Control Anual)"
        justificacion = "La probabilidad es baja, el perfil clínico es mayoritariamente compatible con un paciente sano."

    return int(pred_class), f"{pred_prob * 100:.2f}%", interpretacion, justificacion
list_new_patients_raw = [
    {
        'age': 65, 'sex': 1, 'cp': 3, 'trestbps': 150, 'chol': 250, 'fbs': 1,
        'restecg': 0, 'thalach': 120, 'exang': 1, 'oldpeak': 3.0, 'slope': 0,
        'ca': 2, 'thal': 7
    },
    {
        'age': 35, 'sex': 0, 'cp': 0, 'trestbps': 110, 'chol': 180, 'fbs': 0,
        'restecg': 1, 'thalach': 180, 'exang': 0, 'oldpeak': 0.0, 'slope': 2,
        'ca': 0, 'thal': 3
    },
    {
        'age': 55, 'sex': 1, 'cp': 1, 'trestbps': 130, 'chol': 220, 'fbs': 0,
        'restecg': 0, 'thalach': 150, 'exang': 0, 'oldpeak': 1.0, 'slope': 1,
        'ca': 1, 'thal': 6
    }
]
results = []
feature_columns = X.columns.tolist()
for i, patient_data in enumerate(list_new_patients_raw):
    pred_class, pred_prob_pct, interpretacion, justificacion = predict_and_interpret_patient(
        rf_model_final, scaler, feature_columns, patient_data
    )
    results.append({
        'ID': i + 1,
        'Clase (0: Sano / 1: Enfermo)': pred_class,
        'Probabilidad Clase 1': pred_prob_pct,
        'Nivel de Riesgo (Interpretación)': interpretacion,
        'Justificación Clínica': justificacion
    })
df_interpretacion_final = pd.DataFrame(results)
print("\n--- INFERENCIA CON INTERPRETACIÓN CLÍNICA (FASE 7) ---")
print(df_interpretacion_final.to_markdown(index=False))


--- INFERENCIA CON INTERPRETACIÓN CLÍNICA (FASE 7) ---
|   ID |   Clase (0: Sano / 1: Enfermo) | Probabilidad Clase 1   | Nivel de Riesgo (Interpretación)         | Justificación Clínica                                                                                                   |
|-----:|-------------------------------:|:-----------------------|:-----------------------------------------|:------------------------------------------------------------------------------------------------------------------------|
|    1 |                              1 | 81.01%                 | ALTO RIESGO (Acción Clínica Recomendada) | La probabilidad supera el 70%, indicando un patrón clínico severo y consistente con la enfermedad.                      |
|    2 |                              0 | 28.61%                 | BAJO RIESGO (Control Anual)              | La probabilidad es baja, el perfil clínico es mayoritariamente compatible con un paciente sano.                         |
|    3 |        